# Cookpad Scrapper

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import snowflake.connector
import pandas as pd
import os

In [ ]:
# URL of the search results page
url = "https://cookpad.com/ng/search/american"

# Send a GET request to fetch the page content
response = requests.get(url)
response.raise_for_status()  # Raise an exception if the request was unsuccessful

# Parse the page content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all card elements with itemProp="url"
recipe_cards = soup.find_all('a', itemprop="url")

# Initialize an empty list to store the URLs
urls = []

# Loop through each card and extract the href attribute (URL)
for card in recipe_cards:
    # Get the URL from the href attribute and append it to the list
    recipe_url = card.get('href')
    if recipe_url:
        # Make sure to construct the full URL (since it may be a relative path)
        full_url = f"https://cookpad.com{recipe_url}" if recipe_url.startswith('/') else recipe_url
        urls.append(full_url)

# Print all the URLs found
print(len(urls))
for url in urls:
    print(url)

In [15]:
def getting_the_ingredients(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the ingredient list
        ingredient_list = soup.find('div', {'id': 'ingredients'}).find_all('li')

        # Loop through the list of ingredients and extract the text
        ingredients = []
        for ingredient in ingredient_list:
            ingredient_text = ingredient.get_text(strip=True)
            ingredients.append(ingredient_text)
            
    else:
        print("Failed to retrieve the page")
    
    indegrients_str = ', '.join(ingredients)
    return indegrients_str
        


In [16]:
def get_the_step(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful

    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all li elements where the id starts with 'step_'
    step_elements = soup.find_all('li', id=re.compile('^step_'))
    
    steps = []

    # Loop through the found step elements and extract the step number and description
    for step_element in step_elements:
        # Extract the step number
        step_number = step_element.find('div', class_='flex-shrink-0').text.strip()

        # Extract the step description (text inside <p> tag)
        step_description = step_element.find('div', dir='auto').text.strip()

        # Print the extracted step data
        
        steps.append(step_description)
        
        # Convert the steps list to a single string with ', ' as the separator
    steps_str = ', '.join(steps)
    
    return steps_str


In [17]:
# URL of the recipe page
def get_title_and_author(url):
    # Send a GET request to fetch the page content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful

    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the title from the <h1> tag
    title = soup.find('h1', class_='break-words text-cookpad-16 xs:text-cookpad-24 lg:text-cookpad-36 font-semibold leading-tight clear-both')

    # Extract the username from the <span> tag inside the <a> tag
    username = soup.find('a', class_='flex gap-x-sm items-center lg:pt-rg').find('span', class_='text-cookpad-14 text-cookpad-gray-700 font-semibold')

    # Print the extracted title and username
    print("Title:", title.get_text(strip=True) if title else "Not found")
    print("Username:", username.get_text(strip=True) if username else "Not found")
    
    return title.get_text(strip=True), username.get_text(strip=True)



In [ ]:
datas = []

for i in urls:
    object = {}
    object['id'] = urls.index(i)
    object['url'] = i
    object['title'], object['author'] = get_title_and_author(i)
    object['ingredients'] = getting_the_ingredients(i)
    object['steps'] = get_the_step(i)
    datas.append(object)

In [3]:
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASS"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse="COMPUTE_WH",
    database="NEWS_API",
    schema="PUBLIC"
    )

d:\document\Hackaton\RAG\RecipeRecommender\env\Lib\site-packages\snowflake\connector\config_manager.py:351: UserWarning: Bad owner or permissions on C:\Users\USER\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


In [ ]:
conn.cursor().execute(
    """
    CREATE OR REPLACE TABLE recipe_data (
    id INTEGER PRIMARY KEY,
    url STRING,
    title STRING,
    author STRING,
    ingredients STRING,
    steps STRING
    );

    """)

In [21]:
for data in datas:
    conn.cursor().execute(
        """
        INSERT INTO recipe_data (id, url, title, author, ingredients, steps)
        VALUES (%s, %s, %s, %s, %s, %s);
        """,
        (data['id'], data['url'], data['title'], data['author'], data['ingredients'], data['steps'])
    )

In [4]:
cur = conn.cursor()
cur.execute("SELECT * FROM recipe_data")

# Fetch all rows from the executed query
rows = cur.fetchall()

# Get the column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Create a pandas DataFrame from the fetched data
recipe_data_df = pd.DataFrame(rows, columns=column_names)

# Display the DataFrame (if using in a script or interactive session)
recipe_data_df.sample(5)


,ID,URL,TITLE,AUTHOR,INGREDIENTS,STEPS
28,28,https://cookpad.com/ng/recipes/10801737-glazed...,Glazed doughnuts,labiba’s edibles,"4 tablespoonmilk, 2 tablespooncoco powder, 4 t...","Add 2tbsp of milk and 2tbsp of condensed milk,..."
17,17,https://cookpad.com/ng/recipes/15897637-fried-...,Fried sweet potatoes with beans porridge,Amina,"Sweet potatoes, Beans, Palm oilnveg.Oil, Seaso...",Peel your sweet potatoes and cut into Julien s...
13,13,https://cookpad.com/ng/recipes/16410200-humita...,Humitas (steamed fresh corn cakes),MJ's Kitchen,"3freshcorn, 2eggs, 2 tbsfish spice, 1 tspcurry...","Carefully remove the corn husk, Wash the husk ..."
2,2,https://cookpad.com/ng/recipes/12566870-americ...,American pancakes,Meerah's Cuisine,"3 cupsflour, 2 tspbaking powder, 2 tspbaking s...",Mix all dry ingredients together in a bowl. Mi...
7,7,https://cookpad.com/ng/recipes/22619220-chicke...,"CHICKEN MARYLAND, CHIPS, SCRAMBLED EGGS AND SA...",Kitchen Alchemy,"4chickenpieces (bone-in, skin-on), 1 cupall-pu...",Rinse and pat dry the chicken thighs.\nIn a sh...


In [13]:
cur.execute(
    """
 CREATE OR REPLACE FUNCTION NEWS_API.public.recipes_chunk(
    id NUMBER, url STRING, title STRING, author STRING, ingredients STRING, steps STRING
)
    RETURNS TABLE (chunk STRING, id NUMBER, url STRING, title STRING, author STRING, ingredients STRING, steps STRING)
    LANGUAGE python
    RUNTIME_VERSION = '3.9'
    HANDLER = 'text_chunker'
    PACKAGES = ('snowflake-snowpark-python', 'langchain')
    AS
$$
from langchain.text_splitter import RecursiveCharacterTextSplitter
import copy
from typing import Optional

class text_chunker:

    def process(self, id: int, url: str, title: str, author: str, ingredients: Optional[str], steps: Optional[str]):
        if ingredients is None:
            ingredients = ""
        if steps is None:
            steps = ""

        text_to_chunk = ingredients + "\\n" + steps  

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 2000,
            chunk_overlap = 300,
            length_function = len
        )
        chunks = text_splitter.split_text(text_to_chunk)
        for chunk in chunks:
            yield (chunk, id, url, title, author, ingredients, steps)
$$;

    """
)


In [14]:
cur.execute(
    """
    CREATE TABLE NEWS_API.public.recipe_description_chunks AS (
    SELECT
        recipes.*,
        t.chunk AS chunk
    FROM NEWS_API.public.recipe_data recipes,
        TABLE(NEWS_API.public.recipes_chunk(recipes.id, recipes.url, recipes.title, recipes.author, recipes.ingredients, recipes.steps)) t
);
"""
)

In [15]:
cur.execute("""
            CREATE CORTEX SEARCH SERVICE NEWS_API.public.recipe_description_service
    ON CHUNK
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 hour'
    AS (
        SELECT *
        FROM NEWS_API.public.recipe_description_chunks
    );

            
            """)

In [16]:
cur.execute(
    """
    ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';
    """
)